In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from textwrap import indent
from IPython.display import Javascript
from liblet import ANTLR, AnnotatedTreeWalker, Tree, pyast2tree, side_by_side
from IPython.display import Latex
from fractions import Fraction
import importlib
from core.arithmetic_manager import ArithManager
from core.util import *
from core.shunting_yard_parser import ShuntingYardParser
from core.arithmetic_manager import ArithManager
from core.util import *
import math



In [3]:
def solve(expr, domain, show_ast = False, show_blocks = False, show_latex = False):
    AM = ArithManager(domain)
    
    ast = AM.shuntingYardExpr2ast(expr)
    if show_ast: display(ast)
        
    blocks = AM.blocks(ast)
    
    if show_blocks: 
        for block in blocks:
            display(block[1])
        
    MEMORY = dict(blocks)
    main_block = blocks[-1][1]
    child_to_calc = None
    current_block = block_id = None

    while blocks and is_calculable(blocks[0][1]): blocks = blocks[1:]

    while blocks:
        block_id, current_block = blocks[0]
        parent_to_calc = AM.prior(current_block)
        tex = AM.latex(main_block, MEMORY)
        if show_latex: display_latex(tex)
        
        parent_to_calc.children =  [Tree({'type': 'atomExpr', 'value': AM.eval(child, MEMORY), 'priority': 0, '_calc': 'last'}, []) 
                                    if is_next_to_calc(child) else child
                                    for child in parent_to_calc.children]

        if is_calculable(current_block):
            current_block = current_block.children[0]
            blocks = blocks[1:]

        MEMORY[block_id] = current_block

    tex = AM.latex(main_block, {})
    if show_latex: display_latex(tex)
    
    return main_block.children[0].root['value']





In [12]:
from core.expression_generator import generate_expression

wrong_expr = []
for _ in range(30):
    expr = generate_expression(3)
    try:
        shunting_res = solve(expr, 'R')
        if exception: 
            wrong_expr.append(expr)
            continue
    except Exception as e:
        print('** EXCEPTION SHUNTING ** ', e)
        continue

    formatted_expr = expr.replace('x', '*').replace(':', '/').replace('^', '**')
    res =  eval(formatted_expr)

    if not math.isclose(res, shunting_res, rel_tol=1e-9, abs_tol=0.0):
        wrong_expr.append(expr)
                
if wrong_expr != []:
    print(f'*** WRONG {len(wrong_expr)} ***\n')
    for i, it in enumerate(wrong_expr):
        print(f' $$$ {i} $$$  ', it)
else:
    print("ALL DONE")

ALL DONE


In [9]:

expr = '- - - - - - 2 x - 80 - 75 / 100 x 98 + 64 + - 87 / 13 x 64 / 80 / 39 ^ 2 ^ -2'
formatted_expr = expr.replace('x', '*').replace(':', '/').replace('^', '**')
real_expr = expr.replace('x', '*').replace(':', '/')
print(real_expr)
res =  round(eval(formatted_expr), 7)
shunt = round(float(solve(expr, 'R', show_latex=False)), 7)
#print(shunt, '\n', res)
if not math.isclose(res, shunt, rel_tol=1e-9, abs_tol=0.0): print("***")

- - - - - - 2 * - 80 - 75 / 100 * 98 + 64 + - 87 / 13 * 64 / 80 / 39 ^ 2 ^ -2


In [ ]:
token = '-3.743464808816791e-05'
if re.match(r'-?[0-9]+\.[0-9]+$', token):
    print("YES")
    

    
10.4**40